In [8]:
import pandas as pd
import requests
import json
import os
import time
import datetime

from dotenv import load_dotenv

load_dotenv()

True

Upcoming meetings

In [44]:
base_url = "https://api.rise-digital.com.au/v2/meetings"

rise_api_key = os.getenv("RISE_API_KEY")

today_date = datetime.datetime.now().strftime("%Y-%m-%d")
two_weeks_ago = (datetime.datetime.now() - datetime.timedelta(days=15)).strftime("%Y-%m-%d")

historical_params = {
    "earliest": two_weeks_ago,
    "latest": today_date,
    "includeTrials": "true"
}

upcoming_params = {
    "earliest": today_date,
    "latest": today_date,
    "includeTrials": "true"
}

headers = {
    "x-api-key": rise_api_key
}

response_historical = requests.get(base_url, headers=headers, params=historical_params)
print(len(response_historical.json()))

response_upcoming = requests.get(base_url, headers=headers, params=upcoming_params)
print(len(response_upcoming.json()))

130
9


In [45]:
upcoming_meetings = response_upcoming.json()

# create a dataframe from the meetings
upcoming_meetings_df = pd.DataFrame(upcoming_meetings)

upcoming_meetings_df = upcoming_meetings_df[upcoming_meetings_df['meetingStage'] == 'FIELDS']

# print the dataframe
print(upcoming_meetings_df.shape)
upcoming_meetings_df.head(10)

(6, 19)


,club,clubId,dayNightTwilight,driversAvailableTime,featureRaceText,lastUpdatedTime,lateScratchingTime,meetingClass,meetingCode,meetingStage,meetingStatus,numberOfRaces,scheduledDate,scheduledDateAsISO8601,state,tab,track,trackId,trials
0,Bathurst,851869BDFC5436D1E055005E00000019,N,2025-06-09T00:30:00+0000,,2025-06-10T22:40:08+0000,2025-06-10T23:00:00+0000,C-CLASS,BH110625,FIELDS,PLANNED,9,11/06/2025,2025-06-11,NSW,True,Bathurst,851869C7F94036D1E055005E00000019,False
1,Bendigo,851869BB9D7236D1E055005E00000019,N,2025-06-09T01:00:00+0000,,2025-06-10T22:30:10+0000,2025-06-10T22:30:00+0000,C-CLASS,BN110625,FIELDS,PLANNED,10,11/06/2025,2025-06-11,VIC,True,Bendigo,851869C7E8A736D1E055005E00000019,False
2,Canberra,851869BDFEB736D1E055005E00000019,N,2025-06-10T22:30:00+0000,,2025-06-10T23:10:05+0000,2025-06-10T23:00:00+0000,QUALIFYING TRIAL,CB110625,FIELDS,PLANNED,3,11/06/2025,2025-06-11,NSW,False,Canberra,851869C7F94436D1E055005E00000019,True
4,Newcastle,851869BDE5E136D1E055005E00000019,T,2025-06-11T04:00:00+0000,,2025-06-11T01:15:15+0000,2025-06-11T04:00:00+0000,ADVANCEMENT TRIAL,NR110625,FIELDS,PLANNED,5,11/06/2025,2025-06-11,NSW,False,Newcastle,851869C7ED1736D1E055005E00000019,True
6,Redcliffe,851869BD405536D1E055005E00000019,N,2025-06-09T23:00:00+0000,,2025-06-11T04:10:09+0000,2025-06-10T22:30:00+0000,C-CLASS,RE110625,FIELDS,PLANNED,10,11/06/2025,2025-06-11,QLD,True,Redcliffe,851869C7F28736D1E055005E00000019,False
7,Redcliffe,851869BD405536D1E055005E00000019,D,2025-06-10T22:30:00+0000,,2025-06-10T23:15:09+0000,2025-06-10T22:30:00+0000,ADVANCEMENT TRIAL,RE110625D,FIELDS,PLANNED,2,11/06/2025,2025-06-11,QLD,False,Redcliffe,851869C7F28736D1E055005E00000019,True


In [46]:
print(meetings_df.scheduledDate.unique())

['10/06/2025']


In [47]:
meetings_df = upcoming_meetings_df.copy()

In [48]:
meeting_codes = meetings_df['meetingCode'].unique()
base_meeting_url = 'https://api.rise-digital.com.au/v2/meetings/{}/races'

races_df = pd.DataFrame()
for meeting_code in meeting_codes:
    meeting_url = base_meeting_url.format(meeting_code)
    response = requests.get(meeting_url, headers=headers)
    # print(response.json())
    races = response.json()
    races_df = pd.concat([races_df, pd.DataFrame(races)])

print(races_df.shape)
race_codes = races_df['raceCode'].unique()


(39, 45)


In [49]:
races_df.columns

Index(['ageRestriction', 'ageSexDescription', 'ageSexTrackRecord',
       'alsoEligible', 'barrierDrawType', 'betTypes', 'blackType', 'claim',
       'claimRestrictionText', 'discretionaryHandicap', 'distance',
       'distanceInLaps', 'fieldSize', 'gait', 'leadTime',
       'marginFirstToSecond', 'marginSecondToThird', 'meetingCode', 'mileRate',
       'monte', 'name', 'nameShort', 'notes', 'numberAcrossFront',
       'overallTime', 'plannedStartTimeLocal', 'plannedStartTimestamp',
       'prizemoney13', 'prizemoney14', 'prizemoneyAll', 'prizemoneyPositions',
       'quarter1', 'quarter2', 'quarter3', 'quarter4', 'raceClass',
       'raceClassRestriction', 'raceCode', 'raceNumber', 'raceStatus',
       'stakes', 'startType', 'startTypeWord', 'stateBred', 'trackCondition'],
      dtype='object')

In [50]:
race_codes

BHC04062504

NameError: name 'BHC04062504' is not defined

In [51]:
race_and_fields_url = 'https://api.rise-digital.com.au/v2/races/{}/raceAndFields'

race_and_fields_df = pd.DataFrame()
for CODE in race_codes:
    request_url = race_and_fields_url.format(CODE)
    response = requests.get(request_url, headers=headers)
    race_and_fields = response.json()
    race_and_fields_df = pd.concat([race_and_fields_df, pd.DataFrame([race_and_fields])])

print(race_and_fields_df.shape)
race_and_fields_df.raceCode.unique()

(39, 46)


array(['BHC11062514', 'BHC11062513', 'BHC11062505', 'BHC11062506',
       'BHC11062512', 'BHC11062504', 'BHC11062503', 'BHC11062516',
       'BHC11062501', 'BNC11062504', 'BNC11062510', 'BNC11062505',
       'BNC11062502', 'BNC11062503', 'BNC11062506', 'BNC11062509',
       'BNC11062508', 'BNC11062507', 'BNC11062511', 'CBQ11062501',
       'CBQ11062503', 'CBQ11062502', 'NRQ11062501', 'NRQ11062504',
       'NRQ11062505', 'NRQ11062502', 'NRQ11062503', 'REC11062501',
       'REC11062502', 'REC11062510', 'REC11062504', 'REC11062503',
       'REC11062511', 'REC11062507', 'REC11062505', 'REC11062508',
       'REC11062509', 'REQ110625D03', 'REA110625D01'], dtype=object)

In [26]:
horse_ids = fields_df['horseId'].unique()
len(horse_ids)

NameError: name 'fields_df' is not defined

In [ ]:
# get horse ID information 

base_horse_url = 'https://api.rise-digital.com.au/v2/horses/{}'

horses_df = pd.DataFrame()
for horse_id in horse_ids:
    horse_url = base_horse_url.format(horse_id)
    response = requests.get(horse_url, headers=headers)
    horse = response.json()
    horses_df = pd.concat([horses_df, pd.DataFrame(horse)])

print(horses_df.shape)

In [122]:
race_and_fields_df.to_csv('race_and_fields_df.csv', index=False)

In [53]:
print(race_and_fields_df.shape)

exploded_df = race_and_fields_df.explode('raceFields').reset_index(drop=True)

print(exploded_df.shape)

# Convert the dictionaries in 'raceFields' to separate columns
race_fields_normalized = pd.json_normalize(exploded_df['raceFields'])

# Drop the original 'raceFields' column and concatenate the normalized columns
result_df = pd.concat([exploded_df.drop(columns=['raceFields']), race_fields_normalized], axis=1)
print(result_df.shape)

(39, 46)
(325, 46)
(325, 92)


In [54]:
# get a list of duplicate columns
duplicate_cols = result_df.columns[result_df.columns.duplicated()]
print(duplicate_cols)

# drop one of the raceCode columns, they are the exact same

### TODO: assess duplicate columns and update names or drop.... 
result_df[['name']]

Index(['gait', 'name', 'raceCode'], dtype='object')


,name,name
0,STUART VALENTINE FOR BLOODSTOCK INSURANCE 3YO ...,MAMMALS REASON
1,STUART VALENTINE FOR BLOODSTOCK INSURANCE 3YO ...,TAKE HIS PLACE
2,STUART VALENTINE FOR BLOODSTOCK INSURANCE 3YO ...,SEVEN TEE TWO
3,STUART VALENTINE FOR BLOODSTOCK INSURANCE 3YO ...,THE SAMURAI
4,STUART VALENTINE FOR BLOODSTOCK INSURANCE 3YO ...,LIMBO LOTTIE
...,...,...
320,MOBILE QUALIFYING TRIAL - BEFORE RACE 1 AT 4.25PM,PRESS STATEMENT
321,MOBILE QUALIFYING TRIAL - BEFORE RACE 1 AT 4.25PM,MAYWYNS KAREN
322,MOBILE QUALIFYING TRIAL - BEFORE RACE 1 AT 4.25PM,THE ULTIMAT COWBOY
323,MOBILE QUALIFYING TRIAL - BEFORE RACE 1 AT 4.25PM,HOT AND TREACHEROUS NZ


In [55]:
seen = {}
new_cols = []

for col in result_df.columns:
    if col in seen:
        seen[col] += 1
        new_cols.append(f"{col}_{seen[col]}")
    else:
        seen[col] = 0
        new_cols.append(col)

result_df.columns = new_cols

In [56]:
result_df = result_df.rename(columns={'gait': 'gaitRace', 'gait_1': 'gaitHorse'})
result_df = result_df.rename(columns={'name': 'raceName', 'name_1': 'horseName'})
if 'raceCode_1' in result_df.columns:
    result_df.drop(columns=['raceCode_1'], inplace=True)

In [57]:
result_df.columns

Index(['ageRestriction', 'ageSexDescription', 'ageSexTrackRecord',
       'alsoEligible', 'barrierDrawType', 'betTypes', 'blackType', 'claim',
       'claimRestrictionText', 'discretionaryHandicap', 'distance',
       'distanceInLaps', 'fieldSize', 'gaitRace', 'leadTime',
       'marginFirstToSecond', 'marginSecondToThird', 'meetingCode', 'mileRate',
       'monte', 'raceName', 'nameShort', 'notes', 'numberAcrossFront',
       'overallTime', 'plannedStartTimeLocal', 'plannedStartTimestamp',
       'prizemoney13', 'prizemoney14', 'prizemoneyAll', 'prizemoneyPositions',
       'quarter1', 'quarter2', 'quarter3', 'quarter4', 'raceClass',
       'raceClassRestriction', 'raceCode', 'raceNumber', 'raceStatus',
       'stakes', 'startType', 'startTypeWord', 'stateBred', 'trackCondition',
       'age', 'barrier', 'breeder', 'breederId', 'claimingPrice', 'class',
       'colour', 'colourId', 'driverBirthDateTime', 'driverConcessionFlag',
       'driverDOB', 'driverGender', 'driverId', 'driverIn

In [33]:
races_df.to_csv('races_df.csv', index=False)
result_df.to_csv('result_df.csv', index=False)

RESULTS

In [58]:
results_url = 'https://api.rise-digital.com.au/v2/races/{}/results'

results_df = pd.DataFrame()
for race_code in race_codes:
    results_url = results_url.format(race_code)
    response = requests.get(results_url, headers=headers)
    results = response.json()
    results_df = pd.concat([results_df, pd.DataFrame(results)])

print(results_df.shape)
results_df.head(10)

(390, 61)


,age,barrier,beatenMargin,breeder,breederId,broodmareSireId,broodmareSireName,claimingPrice,colour,damId,...,trainerDOB,trainerGender,trainerId,trainerInitials,trainerLastName,trainerName,trainerNameShort,trainerPreferredName,trainerTitle,trotterInPacersRace
0,3,Sr1,,"C A Vella, D A Vella, C J Vella",851869C094E536D1E055005E00000019,851869C32B4136D1E055005E00000019,MCARDLE USA,None,BAY,851869C7155D36D1E055005E00000019,...,1985-08-08,M,851869BF0D6C36D1E055005E00000019,A S,Williams,Aaron Williams,A S Williams,Aaron,Mr,False
1,3,Sr2,0.9,"S F Ellul, S Ellul",D38CF457124037A3E055005E00000019,851869C32B4136D1E055005E00000019,MCARDLE USA,None,BAY,851869C721C436D1E055005E00000019,...,1990-01-30,F,851869BF426036D1E055005E00000019,A J,Turnbull,Amanda Turnbull,A J Turnbull,Amanda,Miss,False
2,3,Fr7,2.3,M J (Mathew) Day,851869BCF05E36D1E055005E00000019,851869C3208036D1E055005E00000019,CHRISTIAN CULLEN NZ,None,BAY,851869C40D4336D1E055005E00000019,...,1991-02-14,M,851869BF5CE036D1E055005E00000019,B M,Hewitt,Brad Hewitt,B M Hewitt,Brad,Mr,False
3,3,Fr1,2.7,"D G DeMamiel, N J Carmichael",851869BB6E2236D1E055005E00000019,851869C2D91C36D1E055005E00000019,GRINFROMEARTOEAR USA,None,BAY,851869C35F4536D1E055005E00000019,...,1982-06-03,M,851869BDFBFD36D1E055005E00000019,N A,Turnbull,Nathan Turnbull,N A Turnbull,Nathan,Mr,False
4,3,Fr4,4,T N (Travis) Jude,851869C161CC36D1E055005E00000019,851869C34EEA36D1E055005E00000019,ART MAJOR USA,None,BAY,851869C42BE836D1E055005E00000019,...,1954-04-18,M,851869BDC4AA36D1E055005E00000019,J J,Camilleri,Joe Camilleri,J J Camilleri,Joe,Mr,False
5,3,Fr8,4.5,A J (Amy) Rue,851869BF570536D1E055005E00000019,851869C2E8F736D1E055005E00000019,ROBIN HOOD,None,BAY,851869C743B436D1E055005E00000019,...,1985-03-11,M,851869BEF31636D1E055005E00000019,T G,Rue,Trent Rue,T G Rue,Trent,Mr,False
6,3,Fr6,4.6,"W F Ronan, C Ronan, G M Ronan",851869C1153036D1E055005E00000019,851869C370BD36D1E055005E00000019,METROPOLITAN USA,None,BAY,851869C38B7936D1E055005E00000019,...,1982-06-03,M,851869BDFBFD36D1E055005E00000019,N A,Turnbull,Nathan Turnbull,N A Turnbull,Nathan,Mr,False
7,3,Fr3,6.1,A K (Tony) Ross,851869BE66CF36D1E055005E00000019,851869C53DB536D1E055005E00000019,GOTHIC DREAM (US),None,BAY,851869C79E8336D1E055005E00000019,...,1982-06-03,M,851869BDFBFD36D1E055005E00000019,N A,Turnbull,Nathan Turnbull,N A Turnbull,Nathan,Mr,False
8,3,Fr5,12.1,P A (Paul) Carmody,851869BF0F4336D1E055005E00000019,851869C639D136D1E055005E00000019,ELSU NZ,None,BAY,851869C7620F36D1E055005E00000019,...,1996-12-23,M,851869C0AA6B36D1E055005E00000019,J D,Betts,Jed Betts,J D Betts,Jed,Mr,False
9,3,Fr2,18.6,"R A Booth, J A Booth",851869BF182636D1E055005E00000019,851869C2C67836D1E055005E00000019,VILLAGE JASPER USA,None,BROWN,851869C5BF1136D1E055005E00000019,...,1965-07-17,M,851869BE249136D1E055005E00000019,W T,White,Wayne White,W T White,Wayne,Mr,False


In [36]:
race_and_results_url = 'https://api.rise-digital.com.au/v2/races/{}/raceAndResults'

race_and_results_df = pd.DataFrame()
for race_code in race_codes:
    race_and_results_url = race_and_results_url.format(race_code)
    response = requests.get(race_and_results_url, headers=headers)
    race_and_results = response.json()
    race_and_results_df = pd.concat([race_and_results_df, pd.DataFrame(race_and_results)])

print(race_and_results_df.shape)
race_and_results_df.head(10)



ValueError: All arrays must be of the same length